# 🎬 Animontaz Video Generator (Colab Edition)
Run this notebook to generate anime videos from images using a Cloud GPU (Google Colab T4).

**Instructions:**
1. Click **Runtime** -> **Change runtime type** -> Select **T4 GPU** (or better).
2. Run the cells below in order.
3. Click the public Gradio link (e.g., `https://...gradio.live`) to open the UI.

In [ ]:
# 1. Install Dependencies (Wait for this to finish!)
print("Installing libraries... This takes about 60 seconds.")
!pip install -q diffusers==0.30.0 transformers==4.44.2 accelerate opencv-python gradio>=5.0.0 safetensors einops pillow "numpy<2.0.0" imageio[ffmpeg] huggingface_hub
print("Installation Complete!")

In [ ]:
# 2. Define Animation Logic
import torch
from diffusers import AnimateDiffVideoToVideoPipeline, MotionAdapter, EulerAncestralDiscreteScheduler
from diffusers.utils import export_to_video
from PIL import Image
import numpy as np
import os

class AnimeAnimator:
    def __init__(self):
        print("Initializing AnimeAnimator...")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.dtype = torch.float16 if self.device == "cuda" else torch.float32
        
        print(f"Device: {self.device}, Dtype: {self.dtype}")
        
        try:
            # 1. Load Motion Adapter
            print("Loading Motion Adapter...")
            self.adapter = MotionAdapter.from_pretrained(
                "guoyww/animatediff-motion-adapter-v1-5-2", 
                torch_dtype=self.dtype
            )
            
            # 2. Load Pipeline (DreamShaper)
            print("Loading AnimateDiff Video Pipeline (DreamShaper)...")
            self.pipe = AnimateDiffVideoToVideoPipeline.from_pretrained(
                "Lykon/DreamShaper",
                motion_adapter=self.adapter,
                torch_dtype=self.dtype
            )
            
            # 3. Set Scheduler
            self.pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(
                self.pipe.scheduler.config, 
                timestep_spacing="trailing",
                beta_schedule="linear"
            )
            
            # 4. Optimizations
            print("Enabling VRAM optimizations...")
            self.pipe.enable_vae_slicing()
            if self.device == "cuda":
                self.pipe.enable_model_cpu_offload()
            
            print("Model loaded successfully!")
            
        except Exception as e:
            print(f"Error initializing model: {e}")
            raise e

    def preprocess_image(self, image_path, width=512, height=512):
        image = Image.open(image_path).convert("RGB")
        image = image.resize((width, height), resample=Image.LANCZOS)
        return image

    def generate(self, image_path, prompt, num_frames=24, strength=0.8, guidance_scale=7.5, steps=25, progress_callback=None):
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image not found: {image_path}")

        init_image = self.preprocess_image(image_path)
        video_input = [init_image] * num_frames
        
        positive_prompt = (
            f"{prompt}, masterpiece, best quality, highres, anime style, "
            "subtle breathing, slight head movement, wind blowing hair, "
            "cinematic lighting, detailed background"
        )
        
        negative_prompt = (
            "bad quality, worse quality, low resolution, static, motionless, "
            "deformed, distorted, disfigured, bad anatomy, extra limbs, "
            "watermark, text, error"
        )
        
        print(f"Generating {num_frames} frames with prompt: {prompt}")
        
        generator = torch.Generator(device="cpu").manual_seed(42)
        
        def callback_fn(step, timestep, latents):
            if progress_callback:
                progress_callback(step, steps)
            return latents

        output = self.pipe(
            video=video_input,
            prompt=positive_prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=steps,
            guidance_scale=guidance_scale,
            strength=strength,
            generator=generator
        )
        
        frames = output.frames[0]
        output_path = "output.mp4"
        export_to_video(frames, output_path, fps=8)
        return output_path

In [ ]:
# 3. Launch UI
import gradio as gr
import sys

# Global model instance
animator = None

def load_model():
    global animator
    if animator is None:
        print("Loading AI Model (this may take a minute)...")
        animator = AnimeAnimator()
    return animator

def generate_video(image_path, prompt, progress=gr.Progress()):
    if not image_path:
        return None
    
    try:
        model = load_model()
        if not prompt:
            prompt = "1girl, cute, smiling, looking at viewer, solo, upper body"
            
        # Colab GPU Settings (Better than local CPU)
        num_frames = 24
        steps = 25
        
        def progress_wrapper(step, total):
            progress((step, total), desc=f"Denoising step {step}/{total}")
            
        output_video = model.generate(
            image_path=image_path,
            prompt=prompt,
            num_frames=num_frames, 
            strength=0.8,
            steps=steps,
            progress_callback=progress_wrapper
        )
        return output_video
    except Exception as e:
        raise gr.Error(f"Generation Failed: {str(e)}")

# UI Layout
with gr.Blocks(title="Animontaz Video MVP") as demo:
    gr.Markdown("# 🎬 Animontaz Video Generator (Colab/Cloud)")
    
    if not torch.cuda.is_available():
        gr.Warning("No GPU detected! Please change Runtime Type to GPU.")

    with gr.Row():
        with gr.Column():
            input_image = gr.Image(label="Upload Anime Image", type="filepath", height=400)
            prompt_input = gr.Textbox(label="Prompt", placeholder="Describe the character...", lines=2)
            generate_btn = gr.Button("✨ Generate Video", variant="primary", size="lg")
        
        with gr.Column():
            output_video = gr.Video(label="Generated Animation", height=400, autoplay=True)
            
    generate_btn.click(generate_video, inputs=[input_image, prompt_input], outputs=[output_video], api_name="generate_video")

print("Starting Gradio... The link will appear below momentarily.")
print("If it doesn't appear, make sure you are running on a GPU instance.")
demo.launch(share=True, debug=True, allowed_paths=['.'])